In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [2]:
# Load CSV from dataset folder
data = pd.read_csv("dataset/mental_health.csv")
data.head()

,Age,Gender,Country,Education,Marital_Status,Income_Level,Employment_Status,Work_Hours_Per_Week,Remote_Work,Job_Satisfaction,...,Ever_Sought_Treatment,On_Therapy_Now,On_Medication,Trauma_History,Social_Support,Close_Friends_Count,Feel_Understood,Loneliness,Discuss_Mental_Health,Has_Mental_Health_Issue
0,40,Male,USA,Some College,Single,Middle,Full-time,27,No,6,...,1,0,0,0,9,3,4,6,Sometimes,1
1,33,Male,India,Bachelor,Married,Middle,Unemployed,47,No,6,...,0,0,0,0,3,2,7,7,Sometimes,1
2,42,Male,Other,High School,Single,Low,Full-time,53,No,1,...,1,0,0,1,5,3,6,9,Sometimes,1
3,53,Male,Germany,Bachelor,Single,Middle,Unemployed,42,No,10,...,0,1,0,0,1,3,10,5,Never,1
4,32,Female,India,Bachelor,Single,High,Student,13,No,4,...,0,0,0,0,1,5,1,6,Sometimes,1


In [3]:
# Check for missing values
print(data.isnull().sum())

# Drop rows with missing values (simplest for now)
data = data.dropna()

# Quick stats
data.describe()

Age                              0
Gender                           0
Country                          0
Education                        0
Marital_Status                   0
Income_Level                     0
Employment_Status                0
Work_Hours_Per_Week              0
Remote_Work                      0
Job_Satisfaction                 0
Work_Stress_Level                0
Work_Life_Balance                0
Ever_Bullied_At_Work             0
Company_Mental_Health_Support    0
Exercise_Per_Week                0
Sleep_Hours_Night                0
Caffeine_Drinks_Day              0
Alcohol_Frequency                0
Smoking                          0
Screen_Time_Hours_Day            0
Social_Media_Hours_Day           0
Hobby_Time_Hours_Week            0
Diet_Quality                     0
Financial_Stress                 0
Feeling_Sad_Down                 0
Loss_Of_Interest                 0
Sleep_Trouble                    0
Fatigue                          0
Poor_Appetite_Or_Ove

,Age,Work_Hours_Per_Week,Job_Satisfaction,Work_Stress_Level,Work_Life_Balance,Ever_Bullied_At_Work,Sleep_Hours_Night,Caffeine_Drinks_Day,Screen_Time_Hours_Day,Social_Media_Hours_Day,...,Previously_Diagnosed,Ever_Sought_Treatment,On_Therapy_Now,On_Medication,Trauma_History,Social_Support,Close_Friends_Count,Feel_Understood,Loneliness,Has_Mental_Health_Issue
count,10000.00000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.00000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,34.94790,39.657900,5.502700,5.49390,5.489400,0.247400,6.815450,2.023300,7.064590,3.141810,...,0.248000,0.36220,0.191000,0.184300,0.25060,5.459700,4.019300,5.469700,5.449700,0.921600
std,11.15639,10.016711,2.884341,2.88163,2.875572,0.431523,1.377975,1.427432,2.955874,2.254204,...,0.431873,0.48066,0.393109,0.387748,0.43338,2.882986,1.998431,2.872021,2.862322,0.268813
min,18.00000,0.000000,1.000000,1.00000,1.000000,0.000000,3.000000,0.000000,1.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,1.000000,0.000000,1.000000,1.000000,0.000000
25%,26.00000,33.000000,3.000000,3.00000,3.000000,0.000000,5.900000,1.000000,5.000000,1.300000,...,0.000000,0.00000,0.000000,0.000000,0.00000,3.000000,3.000000,3.000000,3.000000,1.000000
50%,34.00000,40.000000,6.000000,6.00000,5.000000,0.000000,6.800000,2.000000,7.000000,3.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,5.000000,4.000000,5.000000,5.000000,1.000000
75%,43.00000,46.000000,8.000000,8.00000,8.000000,0.000000,7.700000,3.000000,9.100000,4.700000,...,0.000000,1.00000,0.000000,0.000000,1.00000,8.000000,5.000000,8.000000,8.000000,1.000000
max,75.00000,74.000000,10.000000,10.00000,10.000000,1.000000,11.000000,8.000000,16.000000,12.000000,...,1.000000,1.00000,1.000000,1.000000,1.00000,10.000000,13.000000,10.000000,10.000000,1.000000


In [6]:
features = ['Work_Hours_Per_Week','Social_Media_Hours_Day', 'Work_Stress_Level','Sleep_Hours_Night', 'Screen_Time_Hours_Day', 'Loneliness', 'Social_Support']  # adjust based on your CSV
target = 'Has_Mental_Health_Issue'  # the column you want to predict

X = data[features]
y = data[target]

In [7]:
le = LabelEncoder()
y = le.fit_transform(y)

# Check label mapping
list(zip(le.classes_, range(len(le.classes_))))

[(np.int64(0), 0), (np.int64(1), 1)]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

/home/athira-v/Mini projects/human battery health/DaisyCrux_Cognovoid/backend/venv/lib/python3.12/site-packages/xgboost/training.py:200: UserWarning: [23:23:50] WARNING: /__w/xgboost/xgboost/src/learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes 

In [11]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.918
              precision    recall  f1-score   support

           0       0.14      0.01      0.02       154
           1       0.92      0.99      0.96      1846

    accuracy                           0.92      2000
   macro avg       0.53      0.50      0.49      2000
weighted avg       0.86      0.92      0.89      2000



In [12]:
pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(le, open("label_encoder.pkl", "wb"))
print("Model and encoder saved!")

Model and encoder saved!


In [14]:
# Example quiz data
sample_input = np.array([[5, 3, 4, 2, 4,5,6]])  # stress, sleep, screen, mood, anxiety

pred = model.predict(sample_input)
print("Predicted mental state:", le.inverse_transform(pred)[0])

Predicted mental state: 1


In [15]:
pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(le, open("label_encoder.pkl", "wb"))